# Try Li Plating Sub Models

In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = "../data/esoh/"
oCV_DIR = "../data/ocv/"
fig_DIR = "../figures/figures_model/"
res_DIR = "../data/results_model/"
%matplotlib widget

In [2]:
parameter_values = get_parameter_values()

In [3]:
parameter_values.search('stripping')

Exchange-current density for stripping [A.m-2]	<function stripping_exchange_current_density_Mohtat2020 at 0x00000167BE1ED670>


In [4]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
spm_li = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "irreversible",
    }
)

spm_li_r = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "reversible",
    }
)

spm_li_pr = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "partially reversible",
    }
)

spme = pybamm.lithium_ion.SPMe(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
spme_li = pybamm.lithium_ion.SPMe(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "irreversible",
    }
)
# spm.print_parameter_info()
param=spm_li.param

In [5]:
cell = 4
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [6]:
parameter_values.update(
    {
      "Negative electrode active material volume fraction": eps_n_data,
      "Positive electrode active material volume fraction": eps_p_data,
      "Positive electrode LAM constant proportional term [s-1]": 1.27152e-07,
      "Negative electrode LAM constant proportional term [s-1]": 1.27272e-06,
      "Positive electrode LAM constant exponential term": 1.1992,
      "Negative electrode LAM constant exponential term": 1.1992,
      "SEI kinetic rate constant [m.s-1]":  4.60788219e-16, #1.08494281e-16 , 
      "EC diffusivity [m2.s-1]": 4.56607447e-19,#8.30909086e-19,
      "SEI growth activation energy [J.mol-1]": 1.87422275e+04,#1.58777981e+04,
    #   "Exchange-current density for plating [A.m-2]":0.008,
      # "Lithium plating transfer coefficient": 0.7,
      # "Dead lithium decay constant [s-1]": 1E-4,
    },
    check_already_exists=False,
)

In [7]:
# c_rate_d = '2C'
# c_rate_c = '2C'
# c_rate_c ='C/5'

In [8]:
pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")
pybamm.set_logging_level("INFO")
pybamm.set_logging_level("VERBOSE")
pybamm.set_logging_level("SPAM")
# experiment = pybamm.Experiment(
#     [
#         (
#          "Charge at "+c_rate_c+" until 4.2V", 
#          "Hold at 4.2V until C/50"
#          )
#     ],
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

experiment = pybamm.Experiment(
    [
        (
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50",
         "Rest for 5 min",
         "Discharge at "+c_rate_d+" until 3V",
         )
    ],
    termination="50% capacity",
#     cccv_handling="ode",
)

In [9]:
sim_long = pybamm.Simulation(spm_li, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=0)
solution1 = sol_long


2022-11-07 14:48:59.101 - [INFO] replace_symbols.process_model(43): Start replacing symbols in Single Particle Model
2022-11-07 14:48:59.102 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(-0x1d160c5eb57163d, Discharge capacity [A.h], children=[], domains={}) (rhs)
2022-11-07 14:48:59.104 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0xa82eb6bf1c289cb, X-averaged negative electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-07 14:48:59.107 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x25be03d789b6cf2e, X-averaged positive electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-07 14:48:59.109 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x4af958adcad46617, X-averaged negative particle concentration, children=[], domains={'primary': ['negative par

In [10]:
# parameter_values.update(
#     {
#     #   "Exchange-current density for plating [A.m-2]":0.001,
#     #   "Lithium plating transfer coefficient": 0.5,
#     #   "Dead lithium decay constant [s-1]": 1E-4,
#         # "Exchange-current density for stripping [A.m-2]": 0.00005,
#     },
#     check_already_exists=False,
# )
# c_rate_c ='1.5C'
# experiment = pybamm.Experiment(
#     [
#         (
#          "Charge at "+c_rate_c+" until 4.2V", 
#          "Hold at 4.2V until C/50"
#          )
#     ],
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

sim_long = pybamm.Simulation(spm_li_r, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=0)
solution2 = sol_long

2022-11-07 14:49:17.150 - [INFO] replace_symbols.process_model(43): Start replacing symbols in Single Particle Model
2022-11-07 14:49:17.151 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(-0x1d160c5eb57163d, Discharge capacity [A.h], children=[], domains={}) (rhs)
2022-11-07 14:49:17.152 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0xa82eb6bf1c289cb, X-averaged negative electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-07 14:49:17.154 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x25be03d789b6cf2e, X-averaged positive electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-07 14:49:17.156 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x4af958adcad46617, X-averaged negative particle concentration, children=[], domains={'primary': ['negative par

In [ ]:
# parameter_values.update(
#     {
#     #   "Exchange-current density for plating [A.m-2]":0.002,
#     #   "Lithium plating transfer coefficient": 0.7,
#       # "Dead lithium decay rate [s-1]": 1E-4,
#     },
#     check_already_exists=False,
# )

sim_long = pybamm.Simulation(spm_li_pr, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=0)
solution3 = sol_long

In [ ]:
t1 =  solution1["Time [s]"].entries
V1 =  solution1["Terminal voltage [V]"].entries
I1 =  solution1["Current [A]"].entries
Q1 = -solution1['Discharge capacity [A.h]'].entries
Q_Li1 = solution1["Loss of capacity to lithium plating [A.h]"].entries
I_Li1 = solution1["X-averaged lithium plating interfacial current density"].entries

In [ ]:
t2 =  solution2["Time [s]"].entries
V2 =  solution2["Terminal voltage [V]"].entries
I2 =  solution2["Current [A]"].entries
Q2 = -solution2['Discharge capacity [A.h]'].entries
Q_Li2 = solution2["Loss of capacity to lithium plating [A.h]"].entries
I_Li2 = solution2["X-averaged lithium plating interfacial current density"].entries

In [ ]:
t3 =  solution3["Time [s]"].entries
V3 =  solution3["Terminal voltage [V]"].entries
I3 =  solution3["Current [A]"].entries
Q3 = -solution3['Discharge capacity [A.h]'].entries
Q_Li3 = solution3["Loss of capacity to lithium plating [A.h]"].entries
I_Li3 = solution3["X-averaged lithium plating interfacial current density"].entries

In [ ]:
linestyles = ["dashed", "dotted", "solid"]
colors = ["tab:purple", "tab:cyan", "tab:red", "tab:green", "tab:blue"]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 8))
ax1 = axes.flat[0]
ax1.plot(t1,V1,'b')
ax1.plot(t2,V2,'r--')
ax1.plot(t3,V3,'g--')
ax1.set_title('Voltage [V]')
ax2 = axes.flat[1]
ax2.plot(t1,Q_Li1,'b')
ax2.plot(t2,Q_Li2,'r--')
ax2.plot(t3,Q_Li3,'g--')
ax2.set_title('Capacity lost due to plating [Ah]')
ax3=axes.flat[2]
ax3.plot(t1,I_Li1,'b')
ax3.plot(t2,I_Li2,'r--')
ax3.plot(t3,I_Li3,'g--')
ax3.set_title('Lithium plating interfacial current density')
# ax1.legend(['baseline',r'decreasing $\alpha_{pl}$',r'Increasing $k_{0,pl}$'])
ax1.legend(['Irreversible ','Reversible','Partially Reversible'])
ax1.legend(['C/5','1.5C'])
fig.tight_layout()
plt.savefig(fig_DIR+'plating_models_parameters'+'.png')

In [ ]:
dasasdasd

In [ ]:
parameter_values.update(
    {
      "Exchange-current density for plating [A.m-2]":0.008,
      "Lithium plating transfer coefficient": 0.7,
    #   "Dead lithium decay constant [s-1]": 1E-4,
        # "Exchange-current density for stripping [A.m-2]": 0.00005,
    },
    check_already_exists=False,
)
pybamm.set_logging_level("WARNING")
experiment = pybamm.Experiment(
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *dfe.N.iloc[-1],
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)
all_sumvars_dict_spm = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)
all_sumvars_dict_spm_li = cycle_adaptive_simulation(spm_li, parameter_values, experiment,SOC_0, save_at_cycles=1)
def plotc2(all_sumvars_dict1,all_sumvars_dict2,esoh_data):
    esoh_vars = ["x_100", "y_0", "C_n", "C_p", "Capacity [A.h]", "Loss of lithium inventory [%]"]
    fig, axes = plt.subplots(3,2,figsize=(7,7))
    for k, name in enumerate(esoh_vars):
        ax = axes.flat[k]
        ax.plot(all_sumvars_dict1["Cycle number"],all_sumvars_dict1[name],"b.")
        ax.plot(all_sumvars_dict2["Cycle number"],all_sumvars_dict2[name],"rv")
        ax.plot(esoh_data["N"],esoh_data[name],"kx")
        ax.set_title(split_long_string(name))
        if k ==2 or k==3:
            ax.set_ylim([3,6.2])
        if k>3:
            ax.set_xlabel("Cycle number")
    fig.legend(["No Plating", "With Plating" , "Data"], 
           loc="lower center",bbox_to_anchor=[0.5,-0.05], ncol=1, fontsize=11)
    fig.tight_layout()
    return fig
plotc2(all_sumvars_dict_spm,all_sumvars_dict_spm_li,dfe)
plt.savefig(fig_DIR+"try_plating_cell_"+cell_no+"_2.png")